# Understanding PLONK (Part 2): Polynomial Encoding

In the previous article, we transformed the "validity check" of a circuit's computation into a set of addition/multiplication constraints.

## Probabilistic Checking of Polynomials
If two polynomials $f(X)$ and $g(X)$ are both of degree at most $d$, the Verifier only needs to provide a random challenge value $\zeta\in \mathbb{F}$ and check whether $f(\zeta)$ equals $g(\zeta)$. This will, with high probability, determine whether $f(X) = g(X)$, with an error probability of $\leq\frac{d}{|\mathbb{F}|}$​. As long as $\mathbb{F}$ is sufficiently large, the probability of error can be negligible.

This result is known as the Schwartz-Zippel lemma.

For example, to verify whether two vectors $\vec{a} + \vec{b}$ equal $\vec{c}$, we first encode the three vectors into polynomials to enable a single-step challenge verification.

A straightforward approach is to encode the vectors as the "coefficients" of the polynomials:

$$
\begin{split}
a(X) &= a_0 + a_1X+a_2X^2 + \cdots + a_{n-1}X^{n-1}\\
b(X) &= b_0 + b_1X+b_2X^2 + \cdots + b_{n-1}X^{n-1}\\
c(X) &= c_0 + c_1X+c_2X^2 + \cdots + c_{n-1}X^{n-1}
\end{split}
$$

Clearly, if $a_i+ b_i = c_i$​, then $a(X)+b(X)=c(X)$. We can then verify this by challenging a random number $\zeta$ and checking the evaluations of the three polynomials at $X=\zeta$:

$$
a(\zeta)+b(\zeta)\overset{?}{=}c(\zeta) 
$$

If the above equation holds, then $\vec{a} + \vec{b}=\vec{c}$.

##  Lagrange Interpolation and the Evaluation Form

If we want to verify $\vec{a}\circ\vec{b}\overset{?}{=}\vec{c}$, the coefficient encoding method becomes less convenient because $a(X)\cdot b(X)$ generates many cross-terms. Moreover, the terms $a_i\cdot b_i$ and $c_i$ do not correspond to the coefficients of $X^i$. For example, the coefficient of $a_1\cdot b_1$ appears in the $X^2$ term, but the coefficient of $X^2$ also includes contributions from the cross-terms $a_0\cdot b_2$ and $a_2\cdot b_0$, while $c_1$ is $X^1$ the coefficient of $X^1$.

We need another polynomial encoding scheme, using the Lagrange basis instead of the monomial basis. If we want to construct a polynomial $a(X)$ such that its evaluations over the domain $H=(w_0, w_1, \ldots w_{N-1})$ match the vector $\vec{a}$, i.e.,

$$
\begin{split}
a(w_0) &= a_0 \\
a(w_1) &= a_1 \\
&\vdots \\
a(w_{N-1}) &= a_{N-1} \\
\end{split}
$$

Interpolation requires a set of basis polynomials: $\{L_i(X)\}_{i\in[0,N-1]}$，where $L_i(w_i)=1$，but $L_i(w_j)=0 (j\neq i)$。The vector $\vec{a}$ can then be encoded as:

$$
a(X)=a_0\cdot L_0(X) + a_1\cdot L_1(X)+ a_2\cdot L_2(X) + \cdots + a_{N-1}\cdot L_{N-1}(X)
$$

A quick mental check shows that when $X=w_0$, all terms except the 0th vanish, so $a(w_0)=a_0$. The polynomials $L_i(X)$ are known as the Lagrange basis polynomials.

We use the same method to encode $b(X)$ and $c(X)$：

$$
\begin{split}
b(X)=b_0\cdot L_0(X) + b_1\cdot L_1(X)+ b_2\cdot L_2(X) + \cdots + b_{N-1}\cdot L_{N-1}(X) \\
c(X)=c_0\cdot L_0(X) + c_1\cdot L_1(X)+ c_2\cdot L_2(X) + \cdots + c_{N-1}\cdot L_{N-1}(X) \\
\end{split}
$$

If $a_i\cdot b_i = c_i$ holds, then $a(w_i)\cdot b(w_i) = c(w_i)$. If $\vec{a}\circ\vec{b}{=}\vec{c}$, then 

$$
a(X)\cdot b(X) = c(X),\quad \forall X\in H
$$

We have now transformed the problem of element-wise vector multiplication into a relationship between three polynomials. The next question is how to perform random challenge verification.

## Random challenge verification

We observe that if the Verifier directly sends a random number $\zeta$ to challenge the above equation, $\zeta$ must belong to $H$. If there is only one $j$ such that $a_j\cdot b_j\neq c_j$​, the probability of the Verifier detecting this error in one challenge is only $\frac{1}{|H|}$​. This would require the Verifier to perform multiple challenges to reduce the error probability, which does not meet our requirements. We want the Verifier to detect Prover cheating with a single challenge.

We can modify the equation by removing the restriction on $X$ and instead use:

$$
a(X)\cdot b(X) - c(X) = q(X)\cdot z_H(X), \quad\forall X\in \mathbb{F}
$$

This equation holds over the entire domain $\mathbb{F}$. Why is this the case?

First, consider the polynomial on the left-hand side: $a(X)\cdot b(X)-c(X)$，which we can define as $f(X)$. We see that $f(X)$ equals zero for $X\in H$, meaning $H$ is precisely the set of roots of $f(X)$. Therefore, $f(X)$ can be factorized as:

$$
f(X)=(X-w_0)(X-w_1)(X-w_2)\cdots(X-w_{N-1})\cdot q(X)
$$

In other words, $f(X)$ is divisible by the polynomial $z_H(X)=(X-w_0)(X-w_1)(X-w_2)\cdots(X-w_{n-1})$, yielding a quotient polynomial $q(X)$ The polynomial $z_H(X)$ is also known as the "vanishing polynomial".

The Prover computes $q(X)$ and sends it to the Verifier; and $H$ is a known system parameter, so the Verifier can compute $z_H(X)$ independently. The Verifier then only needs to perform a single random check to determine whether $a(X)\cdot b(X)-c(X)$ equals zero over $H$:

$$
a(\zeta)\cdot b(\zeta)-c(\zeta) \overset{?}{=} q(\zeta)\cdot z_H(\zeta)
$$

Furthermore, if we use a polynomial commitment scheme, the Verifier can ask the Prover to compute and prove the correctness of these polynomial evaluations at a random challenge point $X = \zeta$, minimizing the Verifier's workload.

However, the Verifier still needs $O(n)$ computations to calculate $z_H(\zeta)$.

Can we further reduce the Verifier's workload? The answer is yes, provided we choose a special $H\subset \mathbb{F}$.


## Roots of Unity

If we choose our evaluation domain $H$ to be the roots of unity, the computational complexity of $z_H(\zeta)$ can be reduced to $O(\log{n})$.

For any finite field $\mathbb{F}_p=(0,1,\ldots,p-1)$, where pp is a prime number, the non-zero elements form a multiplicative group $\mathbb{F}_p^\ast=(1,\ldots,p-1)$ of order $p-1$. Since $p-1$ is always even, its prime factors must include several $2$'s, say $\lambda$ of them. Therefore, $\mathbb{F}_p^\ast$ must contain a multiplicative subgroup of order $2^\lambda$. Let $N=2^{k}, k\leq\lambda$. Then, there exists a multiplicative subgroup of order $N$, denoted as $H$. This subgroup must have a generator, denoted as $\omega$, satisfying $\omega^N = 1$. This is equivalent to taking the $N$-th root of unity, hence the name "roots of unity." However, there is not just one root of unity $\omega$; we find that $\omega^2,\omega^3,\ldots,\omega^{N-1}$ also satisfy the property of roots of unity, i.e., $(\omega^k)^N=1, k\in(2,3,\ldots,N-1)$. All these roots of unity generated by $\omega$ form the multiplicative subgroup $H$:

$$
H=(1,\omega,\omega^2,\omega^3,\ldots,\omega^{N-1})
$$

These elements exhibit certain symmetries: for example, $\omega^{\frac{N}{2}}=-1$ ， $\omega=-\omega^{\frac{N}{2}+1}$ and $\omega^i=-\omega^{\frac{N}{2}+i}$. Additionally, the sum of all roots of unity is zero:

$$
\sum_{i=0}^{N-1}\omega^i=0
$$

As a simple example, consider $\mathbb{F}_{13}$ where we can find a subgroup $H$ of order $4$:

$$
\mathbb{F}_{13}=(0,1,2,3,4,5,6,7,8,9,10,11,12)
$$

The generator of the multiplicative group is $g=2$。Since $13-1=3*2*2$，there exists a multiplicative subgroup of order $4$, with generator $\omega=5$ (all calculations below are $\mod 13$):

$$
H=(\omega^0=1,\omega^1=5,\omega^2=12,\omega^3=8)
$$

And $\omega^4=1=\omega^0$.

In practical applications, we choose a large finite field that contains a large power-of-2 multiplicative subgroup. For example, the scalar field of the elliptic curve `BN254` contains a multiplicative subgroup of order $2^{28}$, and the Scalar Field of `BLS-12-381` contains a multiplicative subgroup of order $2^{32}$.

For the multiplicative subgroup $H$, the following property holds:

$$
z_H(X)=\prod_{i=0}^{N-1}(X-\omega^i)=X^N-1
$$

We can derive this as follow. Suppose $N = 4$. Due to the symmetry $\omega^i=-\omega^{\frac{N}{2}+i}$, the computation can be simplified step by step:

$$
\begin{split}
&(X-\omega^0)(X-\omega^1)(X-\omega^2)(X-\omega^3) \\
=& (X-1)(X-\omega)(X+1)(X+\omega)  \\
=& (X^2-1)(X^2-\omega^2) \\
=& (X^2-1)(X^2+1) \\
=& (X^4-1) \\
\end{split}
$$

## Lagrange Basis
Recall that the Lagrange basis polynomials evaluate to $L_i(w_i)=1$，and $L_i(w_j)=0, (j\neq i)$. Here, we describe how to construct $L_i(X)$'s.

Since $L_i(\omega_j)=0, j \neq i$，it must include the polynomial factor $\prod_{j,j\neq i}(X-\omega_j)$.(In particular, it does *not* include the factor $(X - \omega^i)$.) Now, notice that this expression evaluates to $\prod_{j, j\neq i}(\omega_i-\omega_j)$ at $\omega^i$. To normalize this to equal $1$, we simply divide the expression by this value, yielding the definition of $L_i(X)$:

$$
L_i(X) = \frac{\prod_{j\in H\backslash\{i\}}(X-\omega_j)}{\prod_{j\in H\backslash\{i\}}(\omega_i-\omega_j)} = \prod_{j\in H\backslash\{i\}}^{} \frac{X-\omega_j}{\omega_i-\omega_j}
$$

It is easy to see that $L_i(X)$ equals $1$ at $X=\omega_i$, and $0$ at other points $X=\omega_j, j\neq i$.

Any polynomial $f(X)$ of degree less than $N$ can be uniquely represented as:

$$
f(X)=a_0\cdot L_0(X)+a_1\cdot L_1(X)+a_2\cdot L_2(X)+ \cdots + a_{N-1}\cdot L_{N-1}(X)
$$

We can represent $f(X)$ as its evaluations on $H$, i.e.,$(a_0,a_1,a_2,\ldots,a_{N-1})$. This is called the evaluation form of the polynomial, as opposed to the coefficient form.

The two forms can be converted into each other over $H$ using the (inverse) Fast Fourier Transform algorithm, with a computational complexity of  $O(N\log{N})$.

In [1]:
import numpy as np
import galois

GF13 = galois.GF(13)

# Given two 1-D arrays x and y
# Returns the Lagrange interpolating polynomial through the points (x, y).
def lagrange_poly_in_finite_field(x, y):
    return galois.lagrange_poly(GF13(x), GF13(y))

q_M = [1, 0, 1, 0]

x_points = [1, 5, 12, 8]

# Do the interpolation, return polynomial
q_M_x = lagrange_poly_in_finite_field(x_points, q_M)

print("Output:")
print("Lagrange interpolating polynomial q_M_x: ", q_M_x)

Output:
Lagrange interpolating polynomial q_M_x:  7x^2 + 7


## Polynomial Constraints

Using the Lagrange basis, we can easily impose constraints on various vector computations.

For example, suppose we want to constrain the first element of the vector $\vec{a}=(h,a_1,a_2,\ldots,a_{N-1})$​ to be $v$. We can encode this vector into a polynomial $a(X) and impose the following constraint:

$$
L_0(X)(a(X)-h) = 0, \quad \forall X\in H
$$

Here, $L0(X)$ is the Lagrange basis polynomial corresponding to the first element of the vector. This constraint ensures that $a(X)$ evaluates to $v$ at $X=\omega^0$ (the first element of the domain $H$).

The Verifier can then challenge and verify the following polynomial equation:
$$
L_0(X)(a(X)-h) = q(X)\cdot z_H(X)
$$

### Finite Field Point Interpolation Verification

Given:

$$
L_0(X) = 10X^3 + 10X^2 + 10X + 10 = 10(X^2 + 1)(X + 1)
$$

$$
a(X) = = 7X^2 + 7
$$

$$
q_M = [1, 0, 1, 0]
$$

$$
h = a_0 = q_M[0] = 1
$$

The vanishing polynomial $z_H(X)$（constructed from the roots of unity [1, 5, 12, 8]）:

$$
z_H(X) = (X - 1)(X - 5)(X - 12)(X - 8) = X^4 - 1 = (X^2 + 1)(X + 1)(X - 1)
$$

$$
L_0(X)(a(X)-h) = q(X)\cdot z_H(X)
$$

Now calculate the quotient polynomial $q(X)$:

\begin{aligned}
q(X) &= \frac{L_0(X)(a(X)-h)} {z_H(X)} \\
&= \frac {10(X^2 + 1)(X + 1)(7X^2 + 7 - 1)} {(X^2 + 1)(X + 1)(X - 1)} \\
&= \frac {10(7X^2 + 6)} {(X - 1)} \\
&= \frac {10(7X^2 - 7)} {(X - 1)} \\
&= \frac {70(X + 1)(X - 1)} {(X - 1)} \\
&= \frac {5(X + 1)(X - 1)} {(X - 1)} \\
&= 5(X + 1)
\end{aligned}

The interactive oracle proof is as follows:
1. Prover send $q(X)$ to the Verifier.
2. Verifier constructs $q(X)\cdot z_H(X)$ independently.
3. Verifier randomly selects a challenge $\zeta$ and sends it to the Prover. For the purposes of illustration, suppose $\zeta = 4$.
4. Prover evaluates $L_0(X)(a(X)-h)$ at the challenge point $\zeta$, i.e. $y_1 = L_0(\zeta)(a(\zeta)-h) = 5$，and sends $y_1$ to the Verifier
5. Verifier independently computes $y'_1 = q(\zeta)\cdot z_H(\zeta) = 5$，and checks $y_1 \overset{?}{=} y'_1$.

Below is the code implementation:

In [3]:
import numpy as np
import galois

GF13 = galois.GF(13)

# Given two 1-D arrays x and y
# Returns the Lagrange interpolating polynomial through the points (x, y).
def lagrange_poly_in_finite_field(x, y):
    return galois.lagrange_poly(GF13(x), GF13(y))

q_M = [1, 0, 1, 0]
L_0 = [1, 0, 0, 0]

x_points = [1, 5, 12, 8]

# Do the interpolation, return polynomial
q_M_x = lagrange_poly_in_finite_field(x_points, q_M)
L_0_x = lagrange_poly_in_finite_field(x_points, L_0)

print("Output:")
print("Lagrange interpolating polynomial q_M_x: ", q_M_x)
print("Lagrange interpolating polynomial L_0_x: ", L_0_x)

# Vanishing polynomial: (X - 1)(X - 5)(X - 12)(X - 8)
z_H = galois.Poly([1, 12], field = GF13) * galois.Poly([1, 8], field = GF13) * galois.Poly([1, 1], field = GF13) * galois.Poly([1, 5], field = GF13)

# Quotient polynomial
q = L_0_x * (q_M_x - GF13(1)) // z_H

print("Quotient polynomial q: ", q)
print("Vanishing polynomial z_H: ", z_H)

assert L_0_x * (q_M_x - GF13(1)) == q * z_H, "Division has a remainder(should not have)"

# Use 4 as an example. This number should be be a random number
zeta = 4

y_1_x = L_0_x * (q_M_x - GF13(1))
y_1 = y_1_x(4)
print("y_1: ", y_1)

y_2_x = q * z_H
y_2 = y_2_x(4)
print("y_2: ", y_2)

assert y_1 == y_2, "Check failed"

Output:
Lagrange interpolating polynomial q_M_x:  7x^2 + 7
Lagrange interpolating polynomial L_0_x:  10x^3 + 10x^2 + 10x + 10
Quotient polynomial q:  5x + 5
Vanishing polynomial z_H:  x^4 + 12
y_1:  5
y_2:  5


## More polynomial constraints

### e.g.: boundary conditions
Now suppose we wanted to constrain the first element of the vector $\vec{a}=(a_0,a_1,a_2,\ldots,a_{N-2},a_{N-1})$ to be $a_0 = v_1$，and the last element to be $a_{N-2} = h_2$，and the other elements to be arbitrary. Then $a(X)$ should satisfy the following two constraints:

$$
\begin{split}
L_0(X)\cdot (a(X)-h_1) &= 0, \quad \forall X\in H\\
L_{N-1}(X)\cdot(a(X)-h_2) &= 0, \quad \forall X\in H
\end{split}
$$

Further, by using a random challenge $\alpha$ from the Verifier ($\alpha$), the two constraints above can be merged into a single polynomial constraint:

$$
L_0(X)\cdot (a(X)-h_1) + \alpha\cdot L_{n-1}(X)\cdot(a(X)-h_2) = 0, \quad \forall X\in H
$$

Now, the Verifier need only check this equation at a single random point:

$$
L_0(X)\cdot (a(X)-h_1) + \alpha\cdot L_{n-1}(X)\cdot(a(X)-h_2) = q(X)\cdot z_H(X)
$$

### e.g.: quality except the first element
Now, suppose we wanted to verify that two vectors $\vec{a}$ and $\vec{b}$ of equal length are identical except for the first element. How do we express this as a polynomial constraint? Suppose $a(X)$ and $b(X)$ are the polynomial encodings of the two vectors, then they should satisfy:

$$
(X-\omega^0)(a(X)-b(X))=0
$$

When $X = \omega^0$, the first factor of the polynomial on the left-hand side equals zero. When $X \in H \setminus \{\omega^0\}$, the second factor on the left-hand side equals zero. This means that except for the first element, all other points must have identical values.

As we can see, by using Lagrange polynomials, we can flexibly constrain the relationships between multiple vectors, and we can combine multiple constraints into one, allowing the Verifier to validate multiple vector constraints with just a few random challenges.

## Coset
In the multiplicative group of a finite prime field, for each multiplicative subgroup $H$, there are multiple cosets of the same length. These cosets have properties similar to $H$, and the concept of cosets is also used in Plonk. Here, we will briefly introduce some of their properties.

Taking $\mathbb{F}_{13}$​ as an example, we choose $H=(1,5,12,8)$, and the generator of the multiplicative group is $g=2$. We can then obtain the following two cosets:

\begin{align*}
H_1 &= g\cdot H = (g, g\omega, g\omega^2, g\omega^3) = (2,10,11,3) \\
H_2 &= g^2\cdot H = (g^2, g^2\omega, g^2\omega^2, g^2\omega^3) = (4,7,9,6)
\end{align*}

We can see that $\mathbb{F}^*_{13}=H\cup H_1 \cup H_2$, and their intersection is empty, meaning there is no overlap. Moreover, their Vanishing polynomials can be quickly computed as:

$$
z_{H_1}(X)=X^N-g^N, \quad z_{H_2}(X)=X^N-g^{2N}
$$


## References

- Schwartz–Zippel lemma. https://en.wikipedia.org/wiki/Schwartz%E2%80%93Zippel_lemma
